In [1]:
import os
import sys

import os
import math
import time
import random
import shutil
from pathlib import Path
from contextlib import contextmanager
from collections import defaultdict, Counter

import scipy as sp
import numpy as np
import pandas as pd

from tqdm.auto import tqdm
from functools import partial

import cv2
from PIL import Image

from matplotlib import pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
import albumentations
from albumentations import *
from albumentations.pytorch import ToTensorV2


import timm

from torch.cuda.amp import autocast, GradScaler

import warnings 
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [16]:
IMAGE_SIZE = 640
BATCH_SIZE = 4
TEST_PATH = '../input/ranzcr-clip-catheter-line-classification/test/test'
TRAIN_PATH = '../input/ranzcr-clip-catheter-line-classification/train'
MODEL_PATH = '../input/resnet200d/resnet200d_320_CV9632.pth'

In [14]:
test = pd.read_csv('../input/ranzcr-clip-catheter-line-classification/test/sample_submission.csv')
train = pd.read_csv('../input/ranzcr-clip-catheter-line-classification/train.csv')

In [4]:
class LoadDataset(Dataset):
    def __init__(self, df, transform=None, path = None):
        self.df = df
        self.file_names = df['StudyInstanceUID'].values
        self.transform = transform
        self.path = path
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        labels = torch.tensor(self.df.iloc[idx, 1:12])
        #labels = self.df[target_cols].values
        file_name = self.file_names[idx]
        file_path = f'{self.path}/{file_name}.jpg'
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        return (image, labels)

In [5]:
def get_transforms():
        return Compose([
            Resize(IMAGE_SIZE, IMAGE_SIZE),
            Normalize(
            ),
            ToTensorV2(),
        ])

In [6]:
class ResNet200D(nn.Module):
    def __init__(self, model_name='resnet200d'):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=False)
        n_features = self.model.fc.in_features
        self.model.global_pool = nn.Identity()
        self.model.fc = nn.Identity()
        self.pooling = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(n_features, 11)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        bs = x.size(0)
        features = self.model(x)
        pooled_features = self.pooling(features).view(bs, -1)
        output = self.fc(pooled_features)
        output = self.sigmoid(output)
        return output

ACTUAL TRAINING



In [7]:
#parameters for training
criterion = nn.BCELoss()
n_epochs = 48
lr = 0.00001
device = 'cuda'

In [8]:
#loading data
train_dataset = LoadDataset(train, transform = get_transforms(), path = TRAIN_PATH )
train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True)

In [9]:
model = ResNet200D().to(device)
model.load_state_dict(torch.load(MODEL_PATH)['model'])
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)


In [10]:
def loss(predictions, target, criterion) :
    return criterion(predictions, target)
    

In [11]:
for epoch in range(n_epochs):
    it = 0
    mean_loss = 0
    # Dataloader returns the batches
    for images, labels in tqdm(train_loader):
        if torch.cuda.is_available():
            it+=1
            cur_batch_size = len(images)

            # Flatten the batch of real images from the dataset
            images = images.to(device)
            labels = labels.to(device)

            # Zero out the gradients before backpropagation
            optimizer.zero_grad()

            # forward pass
            predictions  = model(images)

            #loss
            curr_loss = loss(predictions.float(), labels.float(), criterion)

            # Update gradients
            curr_loss.backward(retain_graph=True)

            # Update optimizer
            optimizer.step()

            # Keep track of the average loss
            mean_loss += curr_loss.item()
        
    print("Loss at epoch {} = {}".format(epoch, mean_loss/it))

  0%|          | 0/7521 [00:00<?, ?it/s]

Loss at epoch 0 = 0.18052106640545465


  0%|          | 0/7521 [00:00<?, ?it/s]

Loss at epoch 1 = 0.14769344690646255


  0%|          | 0/7521 [00:00<?, ?it/s]

...

Loss at epoch 45 = 0.005142261394986113


  0%|          | 0/7521 [00:00<?, ?it/s]

Loss at epoch 46 = 0.005504126572738961


  0%|          | 0/7521 [00:00<?, ?it/s]

Loss at epoch 47 = 0.004903125190630294


In [13]:
PATH = '../input'
torch.save(model.state_dict(), "model.pth")

In [ ]:
#do tesint later

In [19]:
class TestDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.file_names = df['StudyInstanceUID'].values
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_name = self.file_names[idx]
        file_path = f'{TEST_PATH}/{file_name}.jpg'
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        return image

In [20]:
def inference(models, test_loader, device):
    tk0 = tqdm(enumerate(test_loader), total=len(test_loader))
    probs = []
    for i, (images) in tk0:
        images = images.to(device)
        avg_preds = []
        for model in models:
            with torch.no_grad():
                y_preds1 = model(images)
                y_preds2 = model(images.flip(-1))
            y_preds = (y_preds1.sigmoid().to('cpu').numpy() + y_preds2.sigmoid().to('cpu').numpy()) / 2
            avg_preds.append(y_preds)
        avg_preds = np.mean(avg_preds, axis=0)
        probs.append(avg_preds)
    probs = np.concatenate(probs)
    return probs

In [ ]:
test_dataset = TestDataset(test, transform=get_transforms())

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4 , pin_memory=True)

In [ ]:
predictions = inference(models, test_loader, device)